In [1]:
# this is a LLM agent that uses aspect based sentiment analysis

In [ ]:
!pip install pyabsa[dev] -U
!pip install torch transformers datasets google-search-results
!pip install python-dotenv
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.2/574.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.

  Preparing metadata (setup.py) ... canceledERROR: Operation cancelled by user
^C


In [2]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


In [3]:
# import relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import json

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, LlamaTokenizer, LlamaForCausalLM
from pyabsa import AspectTermExtraction as ATEPC, available_checkpoints
from serpapi import GoogleSearch
from dotenv import load_dotenv
from openai import OpenAI

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/spacy/cli/_util.py:23: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string
/usr/local/lib/python3.12/dist-packages/weasel/util/config.py:8: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string


[2025-09-12 08:32:26] (2.4.2) PyABSA(2.4.2): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






In [4]:
load_dotenv()
SERPAPIKEY = os.getenv("SERPAPIKEY")
client = OpenAI(api_key=os.getenv("OPENAIAPIKEY"))
CONFIDENCETHRESHOLD = 0.8

In [5]:
modelName = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForSequenceClassification.from_pretrained(modelName)

llmModel = "gpt-4o-mini"

sentimentPipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adecd3f6740>


added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e78ec0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e795c0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e79860>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e79940>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e79a20>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e79b00>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e79be0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e79cc0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e79da0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7adec2e79e80>
Device set to use cuda:0


---
Sentiment & API
---

In [ ]:
# This is an example of basic sentiment analysis

text = "I love this ice-cream"

print(sentimentPipeline(text))

In [ ]:
# This is an example of aspect based sentiment analysis

aspectIceCream = "ice-cream"
aspectCone = "cone"
text = "I love this ice-cream, but the cone is terrible"

inputIce = f"{text} [SEP] {aspectIceCream}"
inputCone = f"{text} [SEP] {aspectCone}"

print("Ice cream sentiment: " + str(sentimentPipeline(inputIce)))
print("Cone sentiment: " + str(sentimentPipeline(inputCone)))

In [ ]:
# We can also use something called pyabsa to handle the different aspects
# Update - This does not work as pyabsa cannot use the model im using

# aspects = ATEPCCheckpointManager.get_aspect_extractor(
#     checkpoint="yangheng/deberta-v3-base-absa-v1.1",
#     auto_device=True # This allows us to use a GPU if one is avaliabel
# )

# text = "I love this ice-cream, but the cone is terrible"

# results = aspects.extract_aspect(
#     inference_source=[text],
#     print_result=True
# )

In [ ]:
# This is code directly from the "yangheng/deberta-v3-base-absa-v1.1" hugging face page that handles aspect extraction

def aspectExtraction(texts):
  aspect_extractor = ATEPC.AspectExtractor(
    'multilingual',
    auto_device=True,
    cal_perplexity=True
  )

  results = aspect_extractor.predict(
    texts,
    print_result=False,
    save_result=False,
    ignore_error=True,
    pred_sentiment=True
  )

  aspectSentiments = []

  for result in results:
    aspects = result['aspect']
    sentiments = result['sentiment']
    confidences = result['confidence']
    sentence = result['sentence']

    for i in range(len(aspects)):
      if confidences[i] > CONFIDENCETHRESHOLD:
        aspectSentiments.append({
            "aspect": aspects[i],
            "sentiment": sentiments[i],
            "sentence": sentence
        })

  return aspectSentiments


In [ ]:
print(aspectSentiments)

In [ ]:
print(result)

In [8]:
# now I am going to attempt to use SerpAPI free tier to get all the google reviews from a certain company
SERPAPIKEY = "800f4e7deaecabcd22b118faadc9f33ac0a65583f5f4e9d1ef5cd9c808cb78ab"

# First we need to get the actual data_id of the location
location = "Swinburne University of Technology"

mapsParameters = {
  "engine": "google_maps",
  "q": location,
  "type": "search",
  "api_key": SERPAPIKEY
}

mapsSearch = GoogleSearch(mapsParameters)
mapsResults = mapsSearch.get_dict()

print(mapsResults)

dataId = mapsResults.get("local_results")[0].get("data_id")

{'search_metadata': {'id': '68c3c8150d0b2910a99dde17', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/a9f0104e80a9f4f1/68c3c8150d0b2910a99dde17.json', 'created_at': '2025-09-12 07:13:25 UTC', 'processed_at': '2025-09-12 07:13:25 UTC', 'google_maps_url': 'https://www.google.com/maps/search/Swinburne+University+of+Technology?hl=en', 'raw_html_file': 'https://serpapi.com/searches/a9f0104e80a9f4f1/68c3c8150d0b2910a99dde17.html', 'total_time_taken': 0.88}, 'search_parameters': {'engine': 'google_maps', 'type': 'search', 'q': 'Swinburne University of Technology', 'google_domain': 'google.com', 'hl': 'en'}, 'search_information': {'local_results_state': 'Results for exact spelling', 'query_displayed': 'Swinburne University of Technology'}, 'local_results': [{'position': 1, 'title': 'Swinburne University of Technology', 'place_id': 'ChIJr1quazJC1moRafeISb1r6XU', 'data_id': '0x6ad642326bae5aaf:0x75e96bbd4988f769', 'data_cid': '8496440632737855337', 'reviews_link': 'https://s

In [ ]:
# Then we need to extract the reviews from google

def getReviews(dataId):
  reviewParameters = {
      "engine": "google_maps_reviews",
      "data_id": dataId,
      "api_key": SERPAPIKEY,
      "hl": "en"
  }

  reviewSearch = GoogleSearch(reviewParameters)
  reviewResults = reviewSearch.get_dict()

  reviewTexts = []

  for review in reviewResults.get("reviews", []):
    reviewTexts.append(review["snippet"])

  return reviewTexts

In [ ]:
for review in reviewResults.get("reviews", []):
  print(f"{review['snippet']}")
  print("-----------------------------------------------------------")

In [ ]:
reviewTexts = []

for review in reviewResults.get("reviews", []):
  reviewTexts.append(review["snippet"])

confidenceThreshold = 0.5

aspects = aspectExtraction(reviewTexts, confidenceThreshold)

for aspect in aspects:
  print(f"aspect: {aspect["aspect"]}")
  print(f"sentiment: {aspect["sentiment"]}")
  print("-----------------------------------------------------------")

In [ ]:
print(aspects)

---
Modular Agent Development
---

This is the tool registry that specifys what tools are avaliable for the agent

below are the tools functions

In [6]:
def getDataID(location):
  mapsParameters = {
    "engine": "google_maps",
    "q": location,
    "type": "search",
    "api_key": SERPAPIKEY
  }

  mapsSearch = GoogleSearch(mapsParameters)
  mapsResults = mapsSearch.get_dict()

  dataId = mapsResults.get("place_results").get("data_id")

  return dataId

In [7]:
print(getDataID("Swinburne University of Technology"))

0x6ad642326bae5aaf:0x75e96bbd4988f769


In [8]:
def getReviews(dataId):
  reviewParameters = {
      "engine": "google_maps_reviews",
      "data_id": dataId,
      "api_key": SERPAPIKEY,
      "hl": "en"
  }

  reviewSearch = GoogleSearch(reviewParameters)
  reviewResults = reviewSearch.get_dict()

  print(reviewResults)

  reviewTexts = []

  for review in reviewResults.get("reviews", []):
    reviewTexts.append(review["snippet"])

  return reviewTexts

In [9]:
print(getReviews("0x6ad642326bae5aaf:0x75e96bbd4988f769"))

{'search_metadata': {'id': '68c3d912ff3b7a167a66cf50', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/338c8c3d75f0cd50/68c3d912ff3b7a167a66cf50.json', 'created_at': '2025-09-12 08:25:54 UTC', 'processed_at': '2025-09-12 08:25:54 UTC', 'google_maps_reviews_url': 'https://www.google.com/maps/place/data=!4m7!3m6!1s0x6ad642326bae5aaf:0x75e96bbd4988f769!5m2!4m1!1i2!9m1!1b1?hl=en&gl=', 'raw_html_file': 'https://serpapi.com/searches/338c8c3d75f0cd50/68c3d912ff3b7a167a66cf50.html', 'prettify_html_file': 'https://serpapi.com/searches/338c8c3d75f0cd50/68c3d912ff3b7a167a66cf50.prettify', 'total_time_taken': 0.62}, 'search_parameters': {'engine': 'google_maps_reviews', 'data_id': '0x6ad642326bae5aaf:0x75e96bbd4988f769', 'hl': 'en'}, 'place_info': {'title': 'Swinburne University of Technology', 'address': 'John St, Hawthorn VIC 3122, Australia', 'rating': 3.9, 'reviews': 743, 'type': 'University'}, 'topics': [{'keyword': 'course', 'mentions': 45, 'id': '/m/07hs4p'}, {'keyword':

In [10]:
def aspectExtraction(texts):
  aspect_extractor = ATEPC.AspectExtractor(
    'multilingual',
    auto_device=True,
    cal_perplexity=True
  )

  results = aspect_extractor.predict(
    texts,
    print_result=False,
    save_result=False,
    ignore_error=True,
    pred_sentiment=True
  )

  aspectSentiments = []

  for result in results:
    aspects = result['aspect']
    sentiments = result['sentiment']
    confidences = result['confidence']
    sentence = result['sentence']

    for i in range(len(aspects)):
      if confidences[i] > CONFIDENCETHRESHOLD:
        aspectSentiments.append({
            "aspect": aspects[i],
            "sentiment": sentiments[i],
            "sentence": sentence
        })

  return aspectSentiments

In [11]:
TOOLREGISTRY = {
  "googleDataIdScraper": {
    "function": getDataID,
    "description": "Accesses google maps with text and returns the dataid of the top result",
    "inputs": {"location": "string"},
    "outputs": {"dataId": "string"},
  },
  "googleReviewScraper": {
    "function": getReviews,
    "description": "Accesses google reviews for a location and returns 8 results",
    "inputs": {"dataId": "string"},
    "outputs": {"reviews": "list<string>"},
  },
  "sentimentAnalysis": {
    "function": aspectExtraction,
    "description": "Performs sentiment analysis on texts and returns aspect based sentiment",
    "inputs": {"texts": "list<string>"},
    "outputs": {"list of dicts with the keys, aspect: string, sentiment: string, and sentence: string"},
  },
}

below is entering an example promps into Llama and recieving a workflow out of it

In [12]:
def runGpt(systemPrompt, userPrompt, model):
  response = client.chat.completions.create(
    model=model,
    messages=[
      {"role": "system", "content": systemPrompt},
      {"role": "user", "content": userPrompt}
    ]
  )
  return response.choices[0].message.content

In [13]:
# prompt engineering tweaking assisted with chatGPT

TOOLSPROMPT = """you are an AI that uses tools and you have access to the following tools:
  1. googleDataIdScraper
    - Description: Accesses Google Maps with text and returns the dataId of the top result.
    - Inputs:
        - location (string, required)
    - Outputs:
        - dataId (string)

  2. googleReviewScraper
    - Description: Accesses Google Reviews for a location and returns 8 results.
    - Inputs:
        - dataId (string, required)
    - Outputs:
        - reviews (list of strings)

  3. sentimentAnalysis
    - Description: Performs aspect-based sentiment analysis on texts.
    - Inputs:
        - texts (list of strings, required)
    - Outputs:
        - results (list of dicts with keys: aspect (string), sentiment (string), sentence (string))

  Before responding, **determine all the tools needed to answer the user’s query**.
  If more than one tool is needed, include them **in order**, so that tools that depend on previous tool outputs come **later in the list**.

  Additionally when there is a input that requires data from a previous tool, please include the tool but do not include the input

  When you need to use a tool, respond ONLY IN JSON using the following format

  [
    {
      "tool": "tool_name",
      "arguments": {
          "input_name_1": "value1",
          "input_name_2": "value2"
      }
    },
        {
      "tool": "tool_name",
      "arguments": {
          "input_name_1": "value1",
          "input_name_2": "value2"
      }
    }
  ]

  respond only in JSON and do not use any additional text
  """

In [14]:
result = runGpt(TOOLSPROMPT, "What are some reviews of Swinburne University of Technology", llmModel)
print(result)
plan = json.loads(result)
print(plan)

[
    {
      "tool": "googleDataIdScraper",
      "arguments": {
          "location": "Swinburne University of Technology"
      }
    },
    {
      "tool": "googleReviewScraper",
      "arguments": {}
    }
]
[{'tool': 'googleDataIdScraper', 'arguments': {'location': 'Swinburne University of Technology'}}, {'tool': 'googleReviewScraper', 'arguments': {}}]


In [15]:
results = {}

for step in plan:
  toolName = step["tool"]
  toolReg = TOOLREGISTRY[toolName]
  arguments = step.get("arguments", {})

  for inputName in toolReg["inputs"]:
    if inputName not in arguments:
      found = False
      for prevTool, prevOutput in results.items():
        if inputName in prevOutput:
          arguments[inputName] = prevOutput[inputName]
          found = True
          break

  output_value = TOOLREGISTRY[toolName]["function"](**arguments)

  if isinstance(toolReg['outputs'], dict):
      output = toolReg['outputs']
  else:
      output = {toolReg['outputs']: output_value}

  results[toolName] = output


print(results)


{'search_metadata': {'id': '68c3d23d73d224fbf827d446', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/d203d0041b3bf5c7/68c3d23d73d224fbf827d446.json', 'created_at': '2025-09-12 07:56:45 UTC', 'processed_at': '2025-09-12 07:56:45 UTC', 'google_maps_reviews_url': 'https://www.google.com/maps/place/data=!4m7!3m6!1sstring!5m2!4m1!1i2!9m1!1b1?hl=en&gl=', 'raw_html_file': 'https://serpapi.com/searches/d203d0041b3bf5c7/68c3d23d73d224fbf827d446.html', 'prettify_html_file': 'https://serpapi.com/searches/d203d0041b3bf5c7/68c3d23d73d224fbf827d446.prettify', 'total_time_taken': 0.82}, 'search_parameters': {'engine': 'google_maps_reviews', 'data_id': 'string', 'hl': 'en'}, 'search_information': {'reviews_results_state': 'Fully empty'}, 'error': "Google hasn't returned any results for this query."}
{'googleDataIdScraper': {'dataId': 'string'}, 'googleReviewScraper': {'reviews': 'list<string>'}}


In [18]:
print(results['googleReviewScraper']['reviews'][1][0])

i
